In [26]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import PIL

In [27]:
model = VGG16(weights='imagenet', include_top=False)

img_path = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/encoded-features/190719_Hela_Ecoli_1to1_01/frame-5.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)

In [28]:
features.shape

(1, 7, 7, 512)

In [29]:
features.flatten().shape

(25088,)

In [30]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model

base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = '/Users/darylwilding-mcbride/Downloads/test images for CNN feature extraction/frame-1.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

In [31]:
block4_pool_features.shape

(1, 14, 14, 512)

In [32]:
block4_pool_features.flatten().shape

(100352,)

In [19]:
# lstm autoencoder recreate sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed

# define input sequence
sequence = np.array([block4_pool_features.flatten()])

# reshape input into [samples, timesteps, features]
n_in = len(sequence)
sequence = sequence.reshape((1, n_in, 100352))

# define model
model = Sequential()
model.add(LSTM(10, activation='relu', input_shape=(None,100352)))
model.add(RepeatVector(n_in))
model.add(LSTM(10, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(sequence, sequence, epochs=300, verbose=0)

# connect the encoder LSTM as the output layer
model = Model(inputs=model.inputs, outputs=model.layers[0].output)

# get the feature vector for the input sequence
yhat = model.predict(sequence)


In [20]:
yhat

array([[1.7524872e-17, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [21]:
n_in

1

In [22]:
sequence.shape

(1, 1, 100352)